In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go

data = pd.read_csv('Coffee-Shop-Sales.csv')

data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='mixed')
data['transaction_time'] = pd.to_datetime(data['transaction_time'], format='%H:%M:%S')
data['revenue'] = data['transaction_qty'] * data['unit_price']


In [19]:
data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='mixed').dt.strftime('%Y-%m-%d')
data_in_month = data.groupby(['transaction_date', 'store_location'], observed=False)['revenue'].sum().unstack().reset_index()

fig = px.line(data_in_month, x='transaction_date', y=data_in_month.columns[1:], 
            title='Pendapatan per Harian di Semua Bulan dan Semua Lokasi',
            labels={'value': 'Pendapatan', 'transaction_date': 'Tanggal', 'variable': 'Lokasi Toko'})
fig.show()

In [9]:
colors = ['turquoise', 'violet', 'wheat']

# Mengelompokkan hasilnya berdasarkan wilayah dan bulan, dan menghitung total pendapatan per bulan
monthly_revenue = data.groupby([data['store_location'], pd.to_datetime(data['transaction_date']).dt.to_period('M')])['revenue'].sum().reset_index()

# Mengonversi objek Period menjadi string dengan format tahun-bulan
monthly_revenue['Bulan'] = monthly_revenue['transaction_date'].dt.strftime('%Y-%m')
monthly_revenue = monthly_revenue.rename(columns={'store_location': 'Lokasi toko'})

# hitung pendapatan
total_revenue_by_location = data.groupby('store_location')['revenue'].sum().reset_index()

# Buat subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=("Pendapatan per Bulan", "Pendapatan keseluruhan"), specs=[[{'type': 'bar'}, {'type': 'domain'}]])

# Tambahkan bar plot ke subplot pertama
for i, lokasi in enumerate(monthly_revenue['Lokasi toko'].unique()):
    bulan = monthly_revenue[monthly_revenue['Lokasi toko'] == lokasi]['Bulan']
    pendapatan = monthly_revenue[monthly_revenue['Lokasi toko'] == lokasi]['revenue']
    fig.add_trace(go.Bar(x=bulan, y=pendapatan, name=lokasi, marker_color=colors[i], hovertemplate='<b>%{label}</b><br>Pendapatan: %{value}<br>'), row=1, col=1)

# Tambahkan pie chart ke subplot kedua
fig.add_trace(go.Pie(labels=total_revenue_by_location['store_location'], values=total_revenue_by_location['revenue'], 
                    marker_colors=colors, hovertemplate='<b>%{label}</b><br>Pendapatan: %{value}<br>'), 
              row=1, col=2)

fig.update_layout(title='Pendapatan per bulan dan keseluruhan pada masing-masing toko')
fig.show()


In [ ]:

# Buat kategori waktu berdasarkan rentang jam
bins = [6, 12, 17, 18, 24]
labels = ['Pagi', 'Siang', 'Sore', 'Malam']
data['time_category'] = pd.cut(data['transaction_time'].dt.hour, bins=bins, labels=labels, right=False)

# Melakukan pengelompokan data
visitor_data = data.groupby(['store_location', 'time_category'], observed=False).agg({'revenue': 'sum', 'transaction_id': 'count'}).reset_index()
visitor_data.columns = ['store_location', 'time_category', 'revenue', 'visitor_count']

# Membuat subplot
fig = make_subplots(rows=1, cols=3, subplot_titles=['Astoria', 'Lower Manhattan', "Hell's Kitchen"], specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]])

# Loop untuk menambahkan pie chart pada setiap subplot
for i, location in enumerate(['Astoria', 'Lower Manhattan', "Hell's Kitchen"], start=1):
    location_data = visitor_data[visitor_data['store_location'] == location]
    fig.add_trace(go.Pie(labels=location_data['time_category'], values=location_data['revenue'],
                         hovertemplate='<b>Waktu</b>: %{label}<br><b>Pendapatan</b>: %{value}<br><b>Pembeli</b>: %{text}',
                         text=location_data['visitor_count'], name=location),
                  row=1, col=i)

fig.update_layout(title='Analisis pembeli datang Berdasarkan Waktu di semua wilayah',
                  showlegend=True)
fig.show()

In [8]:
# Kelompokkan data berdasarkan kategori produk dan wilayah, dan hitung total penjualan
popular_products = data.groupby(['product_category', 'store_location'])['transaction_qty'].sum().reset_index()

# Sorting data berdasarkan total penjualan secara descending untuk menemukan kategori produk paling populer
popular_products = popular_products.sort_values(by='transaction_qty', ascending=False)

# Inisialisasi subplot
fig = make_subplots(rows=1, cols=3, subplot_titles=['Astoria', 'Lower Manhattan', "Hell's Kitchen"], specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]])

# Loop untuk menambahkan pie chart pada setiap subplot
for i, location in enumerate(['Astoria', 'Lower Manhattan', "Hell's Kitchen"], start=1):
    location_data = popular_products[popular_products['store_location'] == location]
    fig.add_trace(go.Pie(labels=location_data['product_category'], values=location_data['transaction_qty'],
                         hovertemplate='<b>Kategori produk</b>: %{label}<br><b>Total terjual</b>: %{value}'),
                  row=1, col=i)

# Update layout
fig.update_layout(title='Analisa kategori produk paling banyak dibeli di setiap lokasi',
                  showlegend=True)
fig.show()
